# Profiling compilation

Because the C++ compiler wants to do more at compile time, and less at run time, the compilation can be long and requires much memory. It makes sense to profile it.

## Globally, for a single-file program

If you want to measure the compilation time of a given code, and this code fits in a single file, you may get it with BASH `time`, GNU `time` or `hyperfine`, as described in the previous notebook.

If you do not care about the absolute compilation time, but rather want to compare two (or more) alternative implementations, you can try [BuildBench](https://build-bench.com/), which lets you easily invesgate different compilers and options.

UNDER WORK : for what concerns the memory used during compilation, we suspect the result of GNU time, as would be for any such basic tool, is wrong because `g++` is actually a frontend and fork children processes for the different compilation steps, and those subprcesses are not taken into account by the timing tool.

In [ ]:
%%file tmp.fibo.cpp

#include <iostream>

constexpr int fibonacci( int n ) {
  if (n>1) return fibonacci(n-1) + fibonacci(n-2) ;
  else return n ;
}

int main() {
  constexpr int val = fibonacci(35) ;
  std::cout << fibonacci(35) << std::endl ;
  return 0 ;
}

In [ ]:
!time -f "%U s" g++ -O2 -std=c++17 tmp.fibo.cpp -o tmp.fibo.exe

In [ ]:
!./tmp.fibo.exe

## Within a file

The compilers have dedicated options, which helps you to know what goes inside:
- `g++ -ftime-report` let you known the time spent in different phases like preprocessing, compilation, assembly, and linking,
- `clang++ -ftime-trace` goes further and produces a json flamegraph.

In [ ]:
!g++ -ftime-report -std=c++17 tmp.fibo.cpp -o tmp.fibo.exe

In [ ]:
!clang++ -ftime-trace -std=c++17 -fconstexpr-steps=100000000 tmp.fibo.cpp -o tmp.fibo2.exe

## When there are many files

As soon as the program is splitted in several modules, if you do not want to handle yourself many numbers, you will need the help of your build system, and a tool such as [Crofiler](https://github.com/HadrienG2/crofiler/) may help.

# Questions ?

# Exercice

Compare the compilation time of the three implementations of fibonacci below, with BASH time, GNU time, hyperfine and/or BuildBench. What do you think ?

In [ ]:
%%file tmp.fibo1.cpp

#include <iostream>

template<int N>
struct fibonacci {
  static int const value = fibonacci<N-1>::value + fibonacci<N-2>::value ;
} ;

template<>
struct fibonacci<1> {
  static int const value = 1 ;
} ;

template<>
struct fibonacci<0> {
  static int const value = 0 ;
} ;

int main() {
  std::cout << fibonacci<35>::value << std::endl ;
  return 0 ;
}

In [ ]:
!g++ -O2 -std=c++20 tmp.fibo1.cpp -o tmp.fibo1.exe

In [ ]:
!./tmp.fibo1.exe

In [ ]:
%%file tmp.fibo2.cpp

#include <iostream>

consteval int fibonacci( int n ) {
  if (n>1) return fibonacci(n-1) + fibonacci(n-2) ;
  else return n ;
}

int main() {
  std::cout << fibonacci(35) << std::endl ;
  return 0 ;
}

In [ ]:
!g++ -O2 -std=c++20 -fconstexpr-ops-limit=333333333 tmp.fibo2.cpp -o tmp.fibo2.exe

In [ ]:
!./tmp.fibo2.exe

In [ ]:
%%file tmp.fibo3.cpp

#include <iostream>
#include <vector>

consteval int fibonacci( int n ) {
  std::vector<int> values(n+1) ;
  values[0] = 0 ;
  values[1] = 1 ;
  for ( int i = 2 ; i <= n ; ++i ) {
    values[i] = values[i-1] + values[i-2] ;
  }
  return values[n] ;
}

int main() {
  std::cout << fibonacci(35) << std::endl ;
  return 0 ;
}

In [ ]:
!g++ -O2 -std=c++20 tmp.fibo3.cpp -o tmp.fibo3.exe

In [ ]:
!./tmp.fibo3.exe

# Resources

- [BuildBench](https://build-bench.com/).
- [Crofiler](https://github.com/HadrienG2/crofiler/)

© *CNRS 2024*
*Assembled and written in french by David Chamont, this work is made available according to the terms of the [Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*